# tennis ball tracker

## packages

In [1]:
import cv2
import numpy as np
from modules import proc

## initialization

In [6]:
video_path = './videos/videoplayback.mp4'
bRecord = False
que = proc.ImgQueue(5)
kernel_size = 30
kernel_dilation = cv2.getStructuringElement(cv2.MORPH_RECT,(kernel_size,kernel_size))
kernel_open = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))

width = 640.

if bRecord == True:
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('{}.avi'.format(video_path.split('/')[-1].split('.')[0]),fourcc, 20.0, (640,360))

## run

In [7]:
cap = cv2.VideoCapture(video_path)

while(cap.isOpened()):
    ret, img = cap.read()
    resize_scale = width / float(img.shape[1])
    img = cv2.resize(img, None, fx=resize_scale, fy=resize_scale)
    img_debug = np.copy(img)
    
    # put img into queue
    img_avg = que.calculate_average().astype(np.uint8)    
    que.enqueue(img)    
    
    # different
    img_diff = np.abs(img.astype(np.float)-img_avg.astype(np.float)).astype(np.uint8)
    img_diff_gray = cv2.cvtColor(img_diff, cv2.COLOR_BGR2GRAY)
    ret, img_diff_gray = cv2.threshold(img_diff_gray, 50, 255, cv2.THRESH_BINARY) 
    img_diff_gray = cv2.morphologyEx(img_diff_gray, cv2.MORPH_CLOSE, kernel_open)
    
    
    # dilation
    img_dilation = cv2.morphologyEx(img_diff_gray, cv2.MORPH_DILATE, kernel_dilation)    
    clusters_dilation = proc.clustering(img_dilation, None, None)
    
    # eliminate players
    for cluster in clusters_dilation:
        if cluster[4] < 1500:
            img_dilation[cluster[1]:cluster[1]+cluster[3], cluster[0]:cluster[0]+cluster[2]] = 0
    img_diff_gray = img_diff_gray - img_dilation    
    ret, img_diff_gray = cv2.threshold(img_diff_gray, 127, 255, cv2.THRESH_BINARY) 
    
    clusters_diff = proc.clustering(img_diff_gray, (2, 30), (0.5, 2.0))
    
    # display
    for rect in clusters_diff:
            cv2.putText(img_debug, 'ball', (rect[0],rect[1]-1), 1, 1.0, (0,255,0),1) 
            cv2.rectangle(img_debug, (rect[0],rect[1]),(rect[0]+rect[2],rect[1]+rect[3]),(0,0,255),2)

    cv2.namedWindow('debug')
    cv2.imshow('debug',img_debug)
    
    # record
    if bRecord == True:
        out.write(img_debug)
        
    # terminate
    k = cv2.waitKey(1) 
    if k == 27:
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        break

cap.release()

if bRecord == True:
    out.release()

cv2.destroyAllWindows()
cv2.waitKey(1)

-1